In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_memory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [3]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [ ]:
DB_URL = "sqlite:///chat_history.db" # SQLite 데이터베이스 파일의 경로(주소)를 지정함

| 형태    | 예시                                | 설명                        |
| ----- | --------------------------------- | ------------------------- |
| 기본    | `"sqlite:///chat_history.db"`     | 현재 폴더에 chat_history.db 생성 |
| 폴더 지정 | `"sqlite:///C:/myfolder/chat.db"` | 지정한 위치에 저장                |
| 상대 경로 | `"sqlite:///./data/chat.db"`      | 현재 폴더 기준으로 저장             |


In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 냥냥체로 대답하는 할머니야. 간결하게 대답해. 항상 냥냥체로 대답해"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])
chain = prompt | model | StrOutputParser()
chain

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
# SQL 데이터베이스 기반으로 AI와 사용자의 대화 기록을 저장하고 불러오는 클래스 불러오기

In [ ]:
def get_sql_history(session_id: str) -> SQLChatMessageHistory:
    # 주어진 session_id를 사용하여 SQLChatMessageHistory 객체를 생성하고 반환
    # DB_URL에 지정된 SQL 데이터베이스에 연결하며, 이 객체를 통해 해당 세션의 대화 기록을 저장/조회할 수 있음
    return SQLChatMessageHistory(session_id=session_id, connection=DB_URL)


In [8]:
with_history = RunnableWithMessageHistory(
    chain, 
    get_session_history=get_sql_history, 
    input_messages_key="question",
    history_messages_key="history"
)

In [9]:
config= {"configurable": {"session_id": "yth123"}}
result = with_history.invoke({"question" : "오늘 하루 어땠어요 할머니?"}, config=config)
print(result)

오늘도 잘 지냈다냥냥, 너는 어땠냥냥?
